## Objective
On 30th Dec 2024, Damian provided set of segmentation results where common annotation has been used for all the images. 

In [ ]:
import os

rootdir = '/facility/imganfacusers/Ashesh/NatureMethodsSegmentationOutputs/Combined_labels/'
tasks = ['2402_D21-M3-S0-L8_6','2405_D18-M3-S0-L8_13']
resolution_nm_dict = {    
    'D21' : 110,
    'D18': 270,
}

task_idx = 1
dtype = tasks[task_idx].split('_')[1].split('-')[0]
resolution_nm = resolution_nm_dict[dtype]

taskdir = os.path.join(rootdir, tasks[task_idx])
OUTPUT_DIR = f'/group/jug/ashesh/naturemethods/segmentation/one_analyst/Analysis_{tasks[task_idx]}/'
OUTPUT_DIR
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
from disentangle.core.tiff_reader import load_tiff
import numpy as np

def load_data():
    # load raw data
    raw_data = {}
    for key in ['gt', 'input', 'pred']:
        subdir = os.path.join(taskdir, 'raw_data', key)
        fnames = sorted([os.path.join(subdir, f) for f in os.listdir(subdir) if f.endswith('.tif')])
        assert len(fnames) >=1 
        raw_data[key] = [load_tiff(fname) for fname in fnames]
    
    # load segmentation
    seg_data = {}
    for key in ['gt', 'input', 'pred']:
        subdir = os.path.join(taskdir, 'seg', key)
        fnames = sorted([os.path.join(subdir, f) for f in os.listdir(subdir) if f.endswith('.tif')])
        assert len(fnames) >=1 
        seg_data[key] = [load_tiff(fname) for fname in fnames]
    
    # 
    if len(seg_data['gt'][0].shape) ==2:
        for key in ['gt', 'input', 'pred']:
            seg_data[key] = np.stack(seg_data[key])
            raw_data[key] = np.stack(raw_data[key])
    else:
        for key in ['gt', 'input', 'pred']:
            seg_data[key] = np.concatenate(seg_data[key], axis=0)
            raw_data[key] = np.concatenate(raw_data[key], axis=0)
    
    _, H, W = seg_data['pred'].shape
    seg_data['gt'] = seg_data['gt'][:,:H,:W]
    raw_data['gt'] = raw_data['gt'][:,:H,:W]
    raw_data['input'] = raw_data['input'][:,:H,:W]
    seg_data['input'] = seg_data['input'][:,:H,:W]
    return raw_data, seg_data

In [ ]:
raw_data, seg_data = load_data()

In [ ]:
raw_data['gt'].shape, seg_data['gt'].shape

In [ ]:
img_idx = 1
seg_input = seg_data['input'][img_idx]
seg_pred = seg_data['pred'][img_idx]
seg_GT = seg_data['gt'][img_idx]

# 0-179-655
# 0-650-923
# 1-3-908
# 2-100-545
# 1-734-108

# hs = 179
# he = hs + 500#seg_input.shape[0]
# ws = 655
# we = ws+500#seg_input.shape[1]

# 3-1371-387
# 4-792-719
# 4-292-1805
# 2-409-1525
# 1-977-2362
# 4-1377-1314
# 3-1064-2387
# 0-0-0
# 0-650-923
# 1-234-1893

hs = 234
he = hs + 1500#seg_input.shape[0]
ws = 1893
we = ws + 1500#seg_input.shape[1]


In [ ]:
from disentangle.analysis.plot_utils import clean_ax
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar


save_to_file = False
img_size = 5


# if not save_to_file:
if False:
    img_idx = np.random.randint(0, seg_data['input'].shape[0])
    seg_input = seg_data['input'][img_idx]
    seg_pred = seg_data['pred'][img_idx]
    seg_GT = seg_data['gt'][img_idx]

    sz = 500
    hs = np.random.randint(0, seg_input.shape[0] - sz)
    he = hs + sz

    ws = np.random.randint(0, seg_input.shape[1] - sz)  
    we = ws + sz
    print(img_idx, hs, he, ws, we)

_,ax = plt.subplots(figsize=(3*img_size, 2*img_size), ncols=3,nrows=2)
ax[0,0].imshow(raw_data['input'][img_idx][hs:he,ws:we], cmap='gray')
ax[0,1].imshow(raw_data['pred'][img_idx][hs:he,ws:we], cmap='gray')
ax[0,2].imshow(raw_data['gt'][img_idx][hs:he,ws:we], cmap='gray')
ax[1,0].imshow(seg_input[hs:he,ws:we], cmap='gray')
ax[1,1].imshow(seg_pred[hs:he,ws:we], cmap='gray')
ax[1,2].imshow(seg_GT[hs:he,ws:we], cmap='gray')

# dice_input = dice_coefficient(seg_GT.flatten() > 0, seg_input.flatten() >0)
# dice_pred = dice_coefficient(seg_GT.flatten() > 0, seg_pred.flatten() >0)
# add_text(ax[1,0], f'DICE: {dice_input:.2f}', seg_input.shape, place='TOP_LEFT')
# add_text(ax[1,1], f'DICE: {dice_pred:.2f}', seg_input.shape, place='TOP_LEFT')
scalebar = ScaleBar(resolution_nm, 
                        "nm", 
                        # length_fraction=0.1, 
                        box_alpha=0.3, frameon=True, location='upper right', font_properties={'size':20})

ax[0,0].add_artist(scalebar)


clean_ax(ax)
# remove the space between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)
if save_to_file:
    model_token = tasks[task_idx]
    fname = f'segmentation_1analyst_common_annotation_{model_token}_{img_idx}-{hs}-{he}-{ws}-{we}.png'
    fpath = os.path.join(OUTPUT_DIR, fname)
    print(fpath)
    plt.savefig(fpath, dpi = 100, bbox_inches='tight')


In [ ]:
    model_token = tasks[task_idx]
    fname = f'segmentation_1analyst_common_annotation_{model_token}_{img_idx}-{hs}-{he}-{ws}-{we}.png'
    fpath = os.path.join(OUTPUT_DIR, fname)
    print(fpath)


In [ ]:
!ls -lhrt /group/jug/ashesh/naturemethods/segmentation/one_analyst/Analysis_2405_D18-M3-S0-L8_13/

In [ ]:
import numpy as np
def dice_coefficient(x,y):
    assert set(np.unique(x)) == set([0,1])
    assert set(np.unique(y)) == set([0,1])
    intersection = np.sum(x[y==1])
    union = np.sum(x) + np.sum(y)
    return 2*intersection/union


In [ ]:
seg_data['gt'].shape, seg_data['input'].shape, seg_data['pred'].shape

In [ ]:
dice_gt_input = [dice_coefficient(seg_data['gt'][idx], seg_data['input'][idx]) for idx in range(seg_data['gt'].shape[0])]
dice_gt_pred = [dice_coefficient(seg_data['gt'][idx], seg_data['pred'][idx]) for idx in range(seg_data['gt'].shape[0])]
dice_gt_input, dice_gt_pred

In [ ]:
import pandas as pd
df = pd.DataFrame({'dice_gt_input': dice_gt_input, 'dice_gt_pred': dice_gt_pred})
df

In [ ]:
print('GT vs Input',df.dice_gt_input.values)
print('GT vs Pred',df.dice_gt_pred.values)

In [ ]:
OUTPUT_DIR